# Installing Libraries

In [ ]:
!pip install -q numpy pandas altair scikit-learn scikit-optimize hyperopt optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.7/107.7 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 5.8 MB/s eta 0:00:00


이번 실습에서는 (1) Exhaustive Search를 위해서 scikit-learn의 [sklearn.model_selection](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.model_selection) 모듈 내에서 제공하고 있는 여러 초매개변수 최적화 방법, (2) Bayesian Optimization을 위해 [scikit-optimize](https://scikit-optimize.github.io/stable/index.html), (3) Tree Parzen Estimator를 위해 [hyperopt](https://hyperopt.github.io/hyperopt/) 및 [optuna](https://optuna.readthedocs.io/en/stable/index.html)를 사용한다.

# Data Preparation

데이터로는 은행의 전화 마케팅 성공 여부를 예측하는 [Banking Dataset - Marketing Targets](https://www.kaggle.com/datasets/prakharrathi25/banking-dataset-marketing-targets)을 사용해보겠다.

In [ ]:
import pandas as pd


URL_TRAIN = 'https://drive.usercontent.google.com/download?export=download&confirm=t&id=1_XXEOnEr8hUqNhFDu8x4Z7Rvb-4ZUTcm'
TRAIN = pd.read_csv(URL_TRAIN, sep=';')
TRAIN

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,technician,married,tertiary,no,825,no,no,cellular,17,nov,977,3,-1,0,unknown,yes
45207,71,retired,divorced,primary,no,1729,no,no,cellular,17,nov,456,2,-1,0,unknown,yes
45208,72,retired,married,secondary,no,5715,no,no,cellular,17,nov,1127,5,184,3,success,yes
45209,57,blue-collar,married,secondary,no,668,no,no,telephone,17,nov,508,4,-1,0,unknown,no


보다시피, 위 데이터는 범주형 데이터와 수치형 데이터가 섞여 있다. One-hot Encoding을 통해서 범주형 데이터를 이진값으로 바꿔주자.

In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder


X, y = TRAIN.drop('y', axis=1), TRAIN['y']
X_NUM, X_CAT = X.select_dtypes(include='number'), X.select_dtypes(exclude='number')

encoder = OneHotEncoder(drop='if_binary', sparse_output=False)
X_CAT_ENC = pd.DataFrame(
    encoder.fit_transform(X_CAT),
    columns=encoder.get_feature_names_out()
)
X_CAT_ENC.head()

,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,...,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


One-hot Encoding된 범주형 데이터와 원래 수치형 데이터를 하나의 데이터로 합쳐주자.

In [ ]:
import pandas as pd


X = pd.concat([X_NUM, X_CAT_ENC], axis=1)
X

,age,balance,day,duration,campaign,pdays,previous,job_admin.,job_blue-collar,job_entrepreneur,...,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown
0,58,2143,5,261,1,-1,0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,44,29,5,151,1,-1,0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,33,2,5,76,1,-1,0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,47,1506,5,92,1,-1,0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,33,1,5,198,1,-1,0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,825,17,977,3,-1,0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
45207,71,1729,17,456,2,-1,0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
45208,72,5715,17,1127,5,184,3,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
45209,57,668,17,508,4,-1,0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


성능 측정치로는 F1 Score를 활용하겠다. Positive Class을 무엇으로 삼을지 확인하기 위해서 레이블 분포를 살펴보자.

In [ ]:
y.value_counts()

y
no     39922
yes     5289
Name: count, dtype: int64

**yes**가 Minority Class이므로, Positive Class로 삼을 것이다. 다음과 같이 yes는 1, no는 0으로 바꿔주자.

In [ ]:
y = y.replace({'no': 0, 'yes': 1})
y.value_counts()

y
0    39922
1     5289
Name: count, dtype: int64

이 데이터를 훈련 데이터셋과 검증 데이터셋으로 분할하자.

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit


splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.4, random_state=42)
I_train, I_test = next(splitter.split(X, y))
X_train, y_train = X.iloc[I_train].values, y.iloc[I_train].values
X_test, y_test = X.iloc[I_test].values, y.iloc[I_test].values

# Default Performance

먼저, Random Seed 이외에 아무런 설정도 하지 않은 기본적인 성능을 확인해보자. 학습 모델로는 Random Forest를 활용해보겠다.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score


model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
f1_score(y_test, y_pred, pos_label=1)

0.4908759124087591

이 성능이 과연 얼마나 개선될 지 다음을 통해 확인해보자.

# Exhaustive Search

먼저, 별도의 알고리즘 없이 일일이 초매개변수를 탐색해보는 방법인 Exhaustive Search 방법을 해보겠다. 물론 Manual Search는 딱히 구현할 필요가 없으므로 Grid Search와 Random Search를 확인해보자.


## Grid Search

초매개변수의 검색 공간을 이산화해서 모든 조합을 찾아보는 Grid Search는 물론, 직접 구현하는 것도 어렵지 않지만 간단하게는 [sklearn.model_selection.GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV)를 활용하면 된다.

먼저, 다음과 같이 검색 공간을 정의하자.

In [ ]:
search_space = {
    'n_estimators': [25, 50, 100, 150], # Decision Tree의 개수
    'criterion': ['gini', 'entropy'], # 불순도 측정 기준
    'max_depth': [3, 5, 10, None], # 각 Decision Tree의 최대 깊이; None일 경우 깊이 제한을 하지 않는다.
    'class_weight': ['balanced', 'balanced_subsample'], # Inbalanced Classification에서 학습한 Cost-sensitive Learning 및 Ensemble 방법이다.
    'min_samples_split': [0.01, 0.1, 0.25, 0.5, 0.75, 1.0], # Decision Tree의 한 노드가 다른 두개의 노드로 분할되기 위한 최소 샘플 수를 의미한다.
                                                            # 예를 들어, 0.01의 경우, 전체 데이터의 0.01만큼 한 노드에 샘플이 존재할 시 두 개의 노드로 분할됨을 의미한다.
}

보다 자세한 초매개변수 설명은 당연히 [API 문서](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier)를 참조하는 것이 더 낫다.

어찌되었든, 총 5종류의 초매개변수에 대해서 서로 다른 값들을 지정해두었다. Grid Search는 각 초매개변수가 가질 수 있는 모든 조합을 다 탐색해보게 된다. 다시 말해서, 4 (n_estimators) * 2 (criterion) * 4 (max_depth) * 2 (class_weight) * 6 (min_samples_split) = 384가지의 초매개변수 조합을 모두 탐색해보는 것이다.

아래와 같이 GridSearchCV를 실행해보자.

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier


optimizer = GridSearchCV(
    estimator=RandomForestClassifier(random_state=42), # 초매개변수를 찾을 학습 모델,
    param_grid=search_space, # 초매개변수 검색 공간,
    scoring='f1', # 성능 측정치; F1 score를 활용한다,
    cv=3, # 내부 CV의 설정; 3를 넣을 시 Stratified 3-Fold CV를 수행한다. 또는, Cross-Validation 시간에 배웠던 Splitter 클래스들을 넣어도 된다.
    n_jobs=-1 # 병렬로 실행할 작업의 개수를 의미한다. -1일 경우 모든 프로세서를 활용한다는 의미다.
)

optimizer.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid={'class_weight': ['balanced', 'balanced_subsample'],
                         'criterion': ['gini', 'entropy'],
                         'max_depth': [3, 5, 10, None],
                         'min_samples_split': [0.01, 0.1, 0.25, 0.5, 0.75, 1.0],
                         'n_estimators': [25, 50, 100, 150]},
             scoring='f1')

많은 시간이 걸릴 것이다. 그도 그럴것이, 384가지의 초매개변수 조합에 내부 CV로 3개의 Split을 훈련/검증하므로 총 1,152번의 학습 모델 훈련이 있기 때문이다.

찾아낸 최적의 초매개변수와 그 성능은 다음과 같이 확인할 수 있다.

In [ ]:
print(f'Best F1 Score: {optimizer.best_score_}')
print('Best Hyperparameters:')
optimizer.best_params_

Best F1 Score: 0.5566594671645814
Best Hyperparameters:


{'class_weight': 'balanced_subsample',
 'criterion': 'entropy',
 'max_depth': None,
 'min_samples_split': 0.01,
 'n_estimators': 50}

그럼, 이 초매개변수들을 가지고 검증 데이터셋에 대한 성능을 확인해보자.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score


model = RandomForestClassifier(
    random_state=42,
    class_weight='balanced_subsample',
    criterion='entropy',
    max_depth=None,
    min_samples_split=0.01,
    n_estimators=50
).fit(X_train, y_train)

y_pred = model.predict(X_test)
f1_score(y_test, y_pred, pos_label=1)

0.5479849231661351

아무 설정도 하지 않은 때보다는 확실히 성능이 증가한 것을 알 수 있다.

또한, 위처럼 직접 초매개변수를 넣어도 되지만, 다음과 같이 최적의 초매개변수를 가진 모델을 가져올 수 있다.

In [ ]:
from sklearn.metrics import f1_score


y_pred = optimizer.best_estimator_.predict(X_test)
f1_score(y_test, y_pred, pos_label=1)

0.5479849231661351

## Random Search

이번엔 무작위로 초매개변수들을 추출하는 Random Search를 해보자. 역시, 직접 구현하는 것도 어렵지 않지만 [sklearn.model_selection.RandomizedSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html#sklearn.model_selection.RandomizedSearchCV)를 사용하는 것이 간편하다.

아래와 같이 검색 공간을 정의하자.

In [ ]:
import scipy.stats as st


search_space = {
    'n_estimators': st.randint(5, 200), # 5 ~ 100 사이의 정수를 균일 분포 확률Uniformly Random로 뽑는다.
    'criterion': ['gini', 'entropy'], # 'gini' 와 'entropy'를 중 하나를 균일 분포 확률Uniformly Random로 뽑는다.
    'max_depth': st.randint(3, 20), # 3 ~ 20 사이의 정수를 균일 분포 확률Uniformly Random로 뽑는다.
    'class_weight': ['balanced', 'balanced_subsample'], # 'balanced' 와 'balanced_subsample'를 중 하나를 균일 분포 확률Uniformly Random로 뽑는다.
    'min_samples_split': st.truncnorm(a=0, b=0.5, loc=0.005, scale=0.01), # 평균은 0.005, 표준 편차는 0.01이, 최소값 0, 최대값 0.5인 정규 분포Normal Distribution에서 뽑는다.
}

이 때, 주의할 점은 검색 공간 정의 시에 [scipy.stats](https://docs.scipy.org/doc/scipy/reference/stats.html#probability-distributions)에서 제공하는 확률 분포 함수 또는 list형을 활용해야 한다는 것이다.

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier


optimizer = RandomizedSearchCV(
    estimator=RandomForestClassifier(random_state=42), # 초매개변수를 찾을 학습 모델,
    param_distributions=search_space, # 초매개변수 검색 공간,
    scoring='f1', # 성능 측정치; F1 score를 활용한다,
    cv=3, # 내부 CV의 설정; 3를 넣을 시 Stratified 3-Fold CV를 수행한다. 또는, Cross-Validation 시간에 배웠던 Splitter 클래스들을 넣어도 된다.
    n_jobs=-1, # 병렬로 실행할 작업의 개수를 의미한다. -1일 경우 모든 프로세서를 활용한다는 의미다.
    n_iter=50 # 몇 회나 초매개변수 공간에서 초매개변수의 조합을 추출할 건지를 정한다. 총 50회를 해보겠다.
)

optimizer.fit(X_train, y_train)

RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(random_state=42),
                   n_iter=50, n_jobs=-1,
                   param_distributions={'class_weight': ['balanced',
                                                         'balanced_subsample'],
                                        'criterion': ['gini', 'entropy'],
                                        'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f9f408137f0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7f9f404370d0>,
                                        'n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f9f79125240>},
                   scoring='f1')

In [ ]:
print(f'Best F1 Score: {optimizer.best_score_}')
print('Best Hyperparameters:')
optimizer.best_params_

Best F1 Score: 0.5673760718744616
Best Hyperparameters:


{'class_weight': 'balanced_subsample',
 'criterion': 'entropy',
 'max_depth': 17,
 'min_samples_split': 0.005781461019160611,
 'n_estimators': 184}

In [ ]:
from sklearn.metrics import f1_score


y_pred = optimizer.best_estimator_.predict(X_test)
f1_score(y_test, y_pred, pos_label=1)

0.5598200899550225

운이 좋게도, 50번의 초매개변수 추출로 Grid Search보다 나은 성능을 냈다. 물론, 매번 그런것은 아닐 것이다.

# Bayesian Optimization

위에서 사용한 두 방법의 장점은 각 초매개변수 조합의 추출이 독립적이므로, 컴퓨팅 자원만 넉넉하다면 수많은 초매개변수들을 한번에 탐색할 수 있다는 것이다. 하지만, 그렇지 않다면 하루 종일 기다려도 끝나지 않을 것이고, 심지어는 (학습 모델에 대한 적절한 지식이 없다면) 쓸데 없는 검색 공간만을 찾고 마무리될 것이다.

이번엔 이전에 탐색한 초매개변수와 성능치 간의 관계를 활용해서 다음의 더 나을 가능성이 높은 초매개변수를 탐색하는 Bayesian Optimization을 활용해보자. 이 방법의 장점은 명확하다. Exhausitive Search 보다는 확실히 적은 컴퓨팅 자원으로도 좋은 초매개변수를 찾을 수 있다.

하지만, 단점또한 명확하다. 이전에 탐색한 초매개변수와 성능치 간의 관계를 활용해야 하므로, 병렬적으로는 실행할 수 없다.

아무튼, Bayesian Optimization은 scikit-optimize의 [skopt.BayesSearchCV](https://scikit-optimize.github.io/stable/modules/generated/skopt.BayesSearchCV.html#skopt.BayesSearchCV)를 활용하면 된다.

## Bayesian Optimization + Gaussian Process
먼저, Surrogate Model을 Gaussian Process로 삼아서 해보자. 그 전에, Gaussian Process와 같은 확률적 회귀 모델이 어떤 출력을 내는 지 확인해보자.

아래와 같이 임의의 연속형 데이터를 만들어보자.



In [ ]:
import numpy as np
import pandas as pd
import altair as alt


gp_X = np.linspace(start=0, stop=10, num=1_000).reshape(-1, 1)
gp_y = np.squeeze(gp_X * np.sin(gp_X))

df_true = pd.DataFrame({'X': np.ravel(gp_X), 'y': gp_y})
alt.Chart(df_true).mark_line().encode(
    x='X:Q', y='y:Q'
)

alt.Chart(...)

그럼, 이제 Gaussian Process 모델이 어떻게 학습되는지를 보겠다. 먼저, 아무런 데이터로 학습이 되지 않았을 때는 어떨까? 다음을 확인해보자.

In [ ]:
import numpy as np
import pandas as pd
import altair as alt
from sklearn.gaussian_process import GaussianProcessRegressor


gp_model = GaussianProcessRegressor()
mean, std = gp_model.predict(gp_X, return_std=True)

df_true = pd.DataFrame({'X': np.ravel(gp_X), 'y': gp_y})
p_true = alt.Chart(df_true).mark_line().encode(x='X:Q', y='y:Q')

df_pred = pd.DataFrame({'X': np.ravel(gp_X), 'y': mean, 'ymin': mean - 1.96 * std, 'ymax': mean + 1.95 * std})
p_pred = alt.Chart(df_pred).mark_line(strokeDash=(8, 8)).encode(x='X:Q', y='y:Q')
p_area = alt.Chart(df_pred).mark_area(opacity=.3).encode(x='X:Q', y='ymin:Q', y2='ymax:Q')
p_area + p_pred + p_true


alt.LayerChart(...)

먼저, Gaussian Process는 예측값과 그 값의 표준 편차를 출력할 수 있다(**predict(return_std=True)**) 다시 말해서, 확률 분포로 표시가 가능하다는 것이다. 위의 그림에서 연한 푸른색으로 된 구역은 95% 신뢰 구간(확률 분포 상에서 95%의 데이터가 해당 범위에 들어 있음)을 의미한다.

아직 아무런 학습이 되지 않았기 때문에, 기본적으로 평균이 0, 표준 편차가 1인 정규 분포로 표현이 되어 있다.

그럼, 일부 데이터만 학습에 쓰게 되면 어떨까?

In [ ]:
import numpy as np
import pandas as pd
import altair as alt
from sklearn.gaussian_process import GaussianProcessRegressor


rnd = np.random.default_rng(42)
I_train = rnd.integers(0, len(gp_y), size=4)

gp_model = GaussianProcessRegressor()
gp_model.fit(gp_X[I_train], gp_y[I_train]) # 임의의 데이터를 훈련 데이터로 활용한다.
mean, std = gp_model.predict(gp_X, return_std=True)

df_true = pd.DataFrame({'X': np.ravel(gp_X), 'y': gp_y})
p_true = alt.Chart(df_true).mark_line().encode(x='X:Q', y='y:Q')

df_pred = pd.DataFrame({'X': np.ravel(gp_X), 'y': mean, 'ymin': mean - 1.96 * std, 'ymax': mean + 1.95 * std})
p_pred = alt.Chart(df_pred).mark_line(strokeDash=(8, 8)).encode(x='X:Q', y='y:Q')
p_area = alt.Chart(df_pred).mark_area(opacity=.3).encode(x='X:Q', y='ymin:Q', y2='ymax:Q')

df_obs = pd.DataFrame({'X': np.ravel(gp_X[I_train]), 'y': gp_y[I_train]})
p_obs = alt.Chart(df_obs).mark_point(filled=True, size=100, color='red').encode(x='X:Q', y='y:Q')

p_area + p_pred + p_true + p_obs

alt.LayerChart(...)

보다시피, 학습 데이터로 활용된 지점(빨간색)에는 95% 신뢰 구간이 없어진 것을 알 수 있다. 또한, 그 인근의 데이터 또한 95% 신뢰 구간이 작아진 것을 확인할 수 있다. 이는 다음과 같은 의미를 가지고 있다:

* 확률 분포로 표현한다는 것의 의미는 아직 관측되지 않은 데이터가 어떠한 분포로 존재하는지를 추정한다는 의미다.
* 이미 관측된 지점은 어떠한 분포로 존재하는지를 알고 있다. 따라서, 95% 신뢰 구간이 없어진 것이다.
* 한 샘플과 인접한 샘플은 서로 큰 관련성이 있다.

이 정도면 확률적 회귀 모델이 Surrogate Model로 어떻게 활용되는지 감이 올 것이다.
* 아직 탐색되지 않은 초매개변수에 대해서 성능치가 어떠한 분포로 존재하는지를 추정한다.
* 이미 탐색한 초매개변수는 어떠한 분포로 존재하는지 알고 있다. 따라서, 95% 신뢰 구간이 없어질 것이다.
* 한 초매개변수와 인접한 초매개변수는 서로 큰 관련성이 있을 것이다.

여기서, **인접한**의 의미를 생각해야 한다. 물론, 당연히 **거리**를 의미하는 것이 맞다. 하지만, Gaussian Process에서는 두 샘플 간의 거리를 측정하는 함수를 **커널Kernel**이라고 한다. SVM에서 배웠던 그 커널과 똑같다. Gaussian Process는 이 커널 함수의 값을 활용해서 한 샘플이 독립적인 확률 분포를 띄는 게 아니라, 다른 샘플과 연관성 있는 확률 분포를 모델링하게 된다.

이론적인 내용은 이정도로 마무리하고, 실제로 초매개변수 최적화에 적용해보자. 먼저, 검색 공간을 정의하자.


In [ ]:
from skopt.space import Categorical, Integer, Real


search_space = {
    'n_estimators': Integer(5, 200), # 5 ~ 200 사이의 정수를 균일 분포 확률Uniformly Random로 뽑는다.
    'criterion': Categorical(['gini', 'entropy']), # 'gini' 와 'entropy'를 중 하나를 균일 분포 확률Uniformly Random로 뽑는다.
    'max_depth': Integer(3, 20), # 3 ~ 20 사이의 정수를 균일 분포 확률Uniformly Random로 뽑는다.
    'class_weight': Categorical(['balanced', 'balanced_subsample']), # 'balanced' 와 'balanced_subsample'를 중 하나를 균일 분포 확률Uniformly Random로 뽑는다.
    'min_samples_split': Real(0.01, 0.5), # 0.01 ~ 0.5 사이의 실수를 균일 분포 확률Uniformly Random로 뽑는다.
}

그 다음에는 Surrogate Model과 Acquisition Function을 정의해야 한다. 보다 자세한 것은 [skopt.Optimizer](https://scikit-optimize.github.io/stable/modules/generated/skopt.Optimizer.html#skopt.Optimizer)를 참조하자.

In [ ]:
optim_params = dict(
    base_estimator='GP', # Gaussian Process를 Surrogate Model로 사용한다
    n_initial_points=10, # 초기 확률 분포를 생성하기 위해 탐색할 초매개변수 조합 수를 의미한다.
                         # 예를 들어, 10으로 설정했다면 초매개변수를 10회 추출한 이후에 Acquisition Function을 활용해 다음의 초매개변수를 탐색한다.
    acq_func='EI', # Expected Improvement를 Acqusition Function으로 사용한다.
    acq_func_kwargs=dict(xi=0.01) # Expected Improvement의 Exploration을 위한 초매개변수인 xi를 의미한다.
)

이제, Bayesian Optimization을 다음과 같이 실행해보자.

In [ ]:
from skopt import BayesSearchCV
from sklearn.ensemble import RandomForestClassifier


optimizer = BayesSearchCV(
    estimator=RandomForestClassifier(random_state=42), # 초매개변수를 찾을 학습 모델,
    search_spaces=search_space, # 초매개변수 검색 공간,
    scoring='f1', # 성능 측정치; F1 score를 활용한다,
    cv=3, # 내부 CV의 설정; 3를 넣을 시 Stratified 3-Fold CV를 수행한다. 또는, Cross-Validation 시간에 배웠던 Splitter 클래스들을 넣어도 된다.
    n_jobs=-1, # 병렬로 실행할 작업의 개수를 의미한다. -1일 경우 모든 프로세서를 활용한다는 의미다.
    n_iter=50, # 몇 회나 초매개변수 공간에서 초매개변수의 조합을 추출할 건지를 정한다. 총 50회를 해보겠다.
    optimizer_kwargs=optim_params # Surrogate Model 및 Acquisition Function의 정의
)

optimizer.fit(X_train, y_train)

BayesSearchCV(cv=3, estimator=RandomForestClassifier(random_state=42),
              n_jobs=-1,
              optimizer_kwargs={'acq_func': 'EI',
                                'acq_func_kwargs': {'xi': 0.01},
                                'base_estimator': 'GP',
                                'n_initial_points': 10},
              scoring='f1',
              search_spaces={'class_weight': Categorical(categories=('balanced', 'balanced_subsample'), prior=None),
                             'criterion': Categorical(categories=('gini', 'entropy'), prior=None),
                             'max_depth': Integer(low=3, high=20, prior='uniform', transform='normalize'),
                             'min_samples_split': Real(low=0.01, high=0.5, prior='uniform', transform='normalize'),
                             'n_estimators': Integer(low=5, high=200, prior='uniform', transform='normalize')})

찾아낸 최적의 초매개변수와 성능은 다음과 같다:

In [ ]:
from sklearn.metrics import f1_score


y_pred = optimizer.best_estimator_.predict(X_test)

print(f'Best F1 Score: {optimizer.best_score_}')
print('Best Hyperparameters:')
print(optimizer.best_params_)
print('F1 Score on Validation set: ', f1_score(y_test, y_pred, pos_label=1))

Best F1 Score: 0.5549388945014703
Best Hyperparameters:


OrderedDict([('class_weight', 'balanced_subsample'),
             ('criterion', 'entropy'),
             ('max_depth', 17),
             ('min_samples_split', 0.01),
             ('n_estimators', 200)])

## Sequential Model-based Algorithm Configuration
scikit-optimize에서는 Gaussian Process 말고도, 다른 확률적 회귀 모델을 Surrogate Model로 활용할 수 있다.


In [ ]:
optim_params = dict(
    base_estimator='RF', # Random Forest를 Surrogate Model로 사용한다; 이 외에도, ET (ExtraTrees), GBRT (Gradient Boosted Tree)를 Surrogate Model로 사용할 수 있다.
    n_initial_points=10, # 초기 확률 분포를 생성하기 위해 탐색할 초매개변수 조합 수를 의미한다.
                         # 예를 들어, 10으로 설정했다면 초매개변수를 10회 추출한 이후에 Acquisition Function을 활용해 다음의 초매개변수를 탐색한다.
    acq_func='EI', # Expected Improvement를 Acqusition Function으로 사용한다.
    acq_func_kwargs=dict(xi=0.01) # Expected Improvement의 Exploration을 위한 초매개변수인 xi를 의미한다.
)

In [ ]:
from skopt import BayesSearchCV
from sklearn.ensemble import RandomForestClassifier


optimizer = BayesSearchCV(
    estimator=RandomForestClassifier(random_state=42), # 초매개변수를 찾을 학습 모델,
    search_spaces=search_space, # 초매개변수 검색 공간,
    scoring='f1', # 성능 측정치; F1 score를 활용한다,
    cv=3, # 내부 CV의 설정; 3를 넣을 시 Stratified 3-Fold CV를 수행한다. 또는, Cross-Validation 시간에 배웠던 Splitter 클래스들을 넣어도 된다.
    n_jobs=-1, # 병렬로 실행할 작업의 개수를 의미한다. -1일 경우 모든 프로세서를 활용한다는 의미다.
    n_iter=50, # 몇 회나 초매개변수 공간에서 초매개변수의 조합을 추출할 건지를 정한다. 총 50회를 해보겠다.
    optimizer_kwargs=optim_params # Surrogate Model 및 Acquisition Function의 정의
)

optimizer.fit(X_train, y_train)

BayesSearchCV(cv=3, estimator=RandomForestClassifier(random_state=42),
              n_jobs=-1,
              optimizer_kwargs={'acq_func': 'EI',
                                'acq_func_kwargs': {'xi': 0.01},
                                'base_estimator': 'RF',
                                'n_initial_points': 10},
              scoring='f1',
              search_spaces={'class_weight': Categorical(categories=('balanced', 'balanced_subsample'), prior=None),
                             'criterion': Categorical(categories=('gini', 'entropy'), prior=None),
                             'max_depth': Integer(low=3, high=20, prior='uniform', transform='normalize'),
                             'min_samples_split': Real(low=0.01, high=0.5, prior='uniform', transform='normalize'),
                             'n_estimators': Integer(low=5, high=200, prior='uniform', transform='normalize')})

In [ ]:
from sklearn.metrics import f1_score


y_pred = optimizer.best_estimator_.predict(X_test)

print(f'Best F1 Score: {optimizer.best_score_}')
print('Best Hyperparameters:')
print(optimizer.best_params_)
print('F1 Score on Validation set: ', f1_score(y_test, y_pred, pos_label=1))

Best F1 Score: 0.5547725709781511
Best Hyperparameters:
OrderedDict([('class_weight', 'balanced_subsample'), ('criterion', 'entropy'), ('max_depth', 20), ('min_samples_split', 0.010042562181452535), ('n_estimators', 199)])
F1 Score on Validation set:  0.5486080746246903


## Tree Parzen Estimator - hyperopt
이번엔 주어진 초매개변수가 낼 성능치를 추정하는 대신, 주어진 성능에서 초매개변수가 속할 확률을 사용하는 Tree Parzen Estimator를 사용해보자. 안타깝게도, scikit-learn 또는 scikit-optimize에서는 제공하지 않는다. 대신, 별도의 라이브러리를 써보자. 바로, [hyperopt](https://hyperopt.github.io/hyperopt)이다.

먼저, 검색 공간을 정의하자.

In [ ]:
from hyperopt import hp


search_space = {
    'n_estimators': 5 + hp.randint('n_estimators', 195), # 5 ~ 200 사이의 정수를 균일 분포 확률Uniformly Random로 뽑는다.
    'criterion': hp.choice('criterion', ['gini', 'entropy']), # 'gini' 와 'entropy'를 중 하나를 균일 분포 확률Uniformly Random로 뽑는다.
    'max_depth': 3 + hp.randint('max_depth', 17), # 3 ~ 20 사이의 정수를 균일 분포 확률Uniformly Random로 뽑는다.
    'class_weight': hp.choice('class_weight', ['balanced', 'balanced_subsample']), # 'balanced' 와 'balanced_subsample'를 중 하나를 균일 분포 확률Uniformly Random로 뽑는다.
    'min_samples_split': hp.uniform('min_samples_split', 0.01, 0.5), # 0.01 ~ 0.5 사이의 실수를 균일 분포 확률Uniformly Random로 뽑는다.
}

이 외에도, 다양한 확률 분포에서 초매개변수를 추출할 수 있다. 보다 자세한 건 다음 문서를 참조하자: [Link](https://hyperopt.github.io/hyperopt/getting-started/search_spaces/)

다음은, 목적 함수를 정의해야 한다. hyperopt에서 목적 함수란, 초매개변수들을 입력으로 받아서 성능 측정치를 출력으로 내놓는 함수를 의미한다. 다음과 같이 정의해보자.

In [ ]:
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold


def objective(hyperparams):
    splitter = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    scores = []

    for I_train, I_test in splitter.split(X_train, y_train):
        X_inner_train, X_inner_test, y_inner_train, y_inner_test = X_train[I_train], X_train[I_test], y_train[I_train], y_train[I_test]
        model = RandomForestClassifier(random_state=42)
        model.set_params(**hyperparams)
        model.fit(X_inner_train, y_inner_train)
        y_pred = model.predict(X_inner_test)
        f1 = f1_score(y_true=y_inner_test, y_pred=y_pred, pos_label=1)
        scores.append(f1)

    return -np.mean(scores)

목적 함수의 출력으로 해줄 평균 F1 Score를 음수로 해준 것을 유념하자. hyperopt는 목적 함수의 값을 항상 **최소화**하는 방향으로만 동작한다.

마지막으로 다음과 같이 최적의 매개변수를 찾아보자.

In [ ]:
from hyperopt import fmin, tpe


best = fmin(
    fn=objective, # 정의한 목적 함수를 입력한다.
    space=search_space, # 검색 공간을 정의한다.
    algo=tpe.suggest, # Tree Parzen Estimator를 활용한다.
    max_evals=50 # 총 50회의 초매개변수 조합을 탐색한다.
)

100%|██████████| 50/50 [04:22<00:00,  5.25s/trial, best loss: -0.5455671095354293]


찾아낸 최적의 초매개변수는 다음과 같다.

In [ ]:
best

{'class_weight': 1,
 'criterion': 1,
 'max_depth': 11,
 'min_samples_split': 0.010751511046357519,
 'n_estimators': 87}

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score


model = RandomForestClassifier(
    random_state=42,
    class_weight='balanced_subsample',
    criterion='entropy',
    max_depth=11,
    min_samples_split=0.010751511046357519,
    n_estimators=87
).fit(X_train, y_train)

y_pred = model.predict(X_test)
f1_score(y_test, y_pred, pos_label=1)

0.5420749279538906

## Tree Parzen Estimator - optuna

TPE를 실행하는 또 다른 방법은 optuna 라이브러리를 활용하는 것이다. hyperopt와 유사하게 optuna도 목적 함수를 정의해야 한다. 하지만, 다른 점은 검색 공간의 정의를 별도로 하는 것이 아니라, 목적 함수 내에서 한다는 것이다.

다음과 같이 목적 함수를 만들어보자.

In [ ]:
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
import optuna
import numpy as np


# optuna에서는 목적 함수의 입력으로 Trial 인스턴스를 받는다.
# Trial은 하나의 초매개변수 후보에 대해서 성능 측정을 하는 단위 과업을 의미한다.
def objective(trial: optuna.Trial):
    # 아래처럼 검색 공간을 정의하고, 초매개변수를 추출한다.
    # 이 추출은 후술할 Sampler에 의해 특정한 확률로 추출된다.
    n_estimators = trial.suggest_int('n_estimators', 5, 200)
    criterion = trial.suggest_categorical('criterion', ['gini', 'entropy'])
    max_depth = trial.suggest_int('max_depth', 3, 20)
    class_weight = trial.suggest_categorical('class_weight', ['balanced', 'balanced_subsample'])
    min_samples_split = trial.suggest_float('min_samples_split', 0.01, 0.05)

    # 여기서부턴 3-Fold CV를 통해 추출한 초매개변수에 대한 성능을 추정한다.
    splitter = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    scores = []

    for I_train, I_test in splitter.split(X_train, y_train):
        X_inner_train, X_inner_test, y_inner_train, y_inner_test = X_train[I_train], X_train[I_test], y_train[I_train], y_train[I_test]
        model = RandomForestClassifier(
            random_state=42,
            n_estimators=n_estimators,
            criterion=criterion,
            max_depth=max_depth,
            class_weight=class_weight,
            min_samples_split=min_samples_split
        )
        model.fit(X_inner_train, y_inner_train)
        y_pred = model.predict(X_inner_test)
        f1 = f1_score(y_true=y_inner_test, y_pred=y_pred, pos_label=1)
        scores.append(f1)

    return np.mean(scores)

그 다음에는 초매개변수를 추출하기 위한 공간을 정의하는 Sampler 인스턴스를 정의해야 한다. 우리는 Tree Parzen Estimator를 사용하므로, [optuna.samplers.TPESampler](https://optuna.readthedocs.io/en/stable/reference/samplers/generated/optuna.samplers.TPESampler.html)를 다음과 같이 정의하자.

In [ ]:
from optuna.samplers import TPESampler


sampler = TPESampler()

이제 최적의 초매개변수를 찾을 차례다.

In [ ]:
import optuna


study = optuna.create_study(
    study_name ='Random Forest with Tree Parzen Estimator', # 이름을 임의로 설정한다.
    sampler=sampler, # 초매개변수를 추출하기 위한 공간을 정의한다.
    direction='maximize', # F1 Score를 최대화하는 초매개변수를 찾는다.
)

optuna.logging.set_verbosity(optuna.logging.WARNING) # 불필요한 로그는 보여주지 말자.

study.optimize(
    func=objective, # 목적 함수를 의미한다
    n_trials=50, # 탐색할 초매개변수들의 개수를 의미한다
    show_progress_bar=True # 표준 출력에 프로그레스 바를 보여준다.
)

  0%|          | 0/50 [00:00<?, ?it/s]

찾아낸 최적의 초매개변수는 다음과 같다.

In [ ]:
study.best_params

{'n_estimators': 171,
 'criterion': 'entropy',
 'max_depth': 15,
 'class_weight': 'balanced_subsample',
 'min_samples_split': 0.01022540757290028}

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score


model = RandomForestClassifier(
    random_state=42,
    **study.best_params
).fit(X_train, y_train)

y_pred = model.predict(X_test)
f1_score(y_test, y_pred, pos_label=1)

0.5475218658892128